In [1]:
%matplotlib inline
%load_ext autoreload
%autoreload 2

import sys,os
sys.path.append('/home/heyangle/Desktop/ScanCraft/ScanCraft')
sys.path.append('/home/vooum/Desktop/ScanCraft/ScanCraft')
sys.path.append('C://Users//vooum//Desktop//ScanCraft//ScanCraft')

# collect all sample documents

In [2]:
import pandas,numpy,copy
from command.file_operations.GetDirectories import GetDirectories
from command.file_operations.GetSamples import GetSamples

In [8]:
# Get all folders from folder_path, folders' name should contain folder_key
folder_path='./samples/data/'
folder_key='data'
folders=GetDirectories(path=folder_path,keyword=folder_key,numbered=True)
folders

['/home/vooum/Desktop/ScanCraft/ScanCraft/laboratory/samples/data/data0',
 '/home/vooum/Desktop/ScanCraft/ScanCraft/laboratory/samples/data/data1']

In [14]:
# get all spectrum files from subfolder(if not exist, set to '') in each folders
subfolder=''
sample_collect=[
    GetSamples( path=os.path.join( folder,subfolder), patterns=['inp','spectr','omega'],number_position=-1 )
    for folder in folders
]
# sample_collect

In [19]:
samples=sum(sample_collect,[])
len(samples)

5302

In [20]:
samples[-1].items()

dict_items([('spectr', '/home/vooum/Desktop/ScanCraft/ScanCraft/laboratory/samples/data/data1/spectr.dat.11554'), ('inp', '/home/vooum/Desktop/ScanCraft/ScanCraft/laboratory/samples/data/data1/inp.dat.11554'), ('omega', '/home/vooum/Desktop/ScanCraft/ScanCraft/laboratory/samples/data/data1/omega.dat.11554')])

## read samples

In [21]:
from command.nexus.NMSSMTools import ReadNToolsOutput
ignore=[ 'Landau Pole'#27
        ,'Excluded by LUX'
        ,'Relic density'
        ,'Muon magn'#37
        ,'b -> c tau nu'#58 always keep alive
        ,'k_bot(H_SM)'
        ,'k_tau(H_SM)'
        ]

In [24]:
sample=ReadNToolsOutput(samples[0]['spectr'],ignore=[])
sample.constraints

['# Muon magn. mom. more than 2 sigma away',
 '# b -> c tau nu more than 2 sigma away (as SM)',
 '# k_bot(H_SM) more than 2 sigma away (as SM)',
 '# k_tau(H_SM) more than 2 sigma away (as SM)']

In [25]:
sample.ignore=ignore
sample.constraints

[]

## chi-square test

In [11]:
from command.Experiments.Chi_square_Test import Chisquare_Test
Chisquare_Test.__dict__.keys()

dict_keys(['__module__', 'mh', 'bsg', 'bmu', 'omg', 'Psd', 'Psi', 'Nsd', '__dict__', '__weakref__', '__doc__'])

In [15]:
from command.Experiments.Chi_square_Test import Chisq
def data_forX2(sample):
    mN=sample('mass',1000022)
    return{
        'mh':sample('mass',25)
        ,'bsg':sample('LOWEN',1)
        ,'bmu':sample('LOWEN',4)
        ,'omg':sample('LOWEN',10)
        ,'Psi':(mN,sample('LOWEN',20))
        ,'Nsd':(mN,sample('LOWEN',30))
        ,'Psd':(mN,sample('LOWEN',40))
    }
data_forX2(sample)

{'mh': 122.924979,
 'bsg': 0.000314714668,
 'bmu': 5.42619385e-09,
 'omg': 0.110258584,
 'Psi': (1099.14054, 0.0),
 'Nsd': (1099.14054, 0.0),
 'Psd': (1099.14054, 0.0)}

In [16]:
Chisq(**data_forX2(sample))

{'mh': 0.47980814134863525,
 'bsg': 3.2561883389202935,
 'bmu': 36.790103736111384,
 'omg': 0.5057421709424116,
 'Psi': 0.0,
 'Nsd': 0.0,
 'Psd': 0.0}

## get values with pool

In [ ]:
from multiprocessing import Pool

In [9]:
def read_spectr(sample):
    spectr_dir=sample['spectr']
    SLHAtext=ReadNToolsOutput(spectr_dir,ignore=ignore)
    #mh=SLHAtext('MASS',25)
    #print(f'{spectr_dir} Read\nMh={mh}')
    calculable=not SLHAtext.error
    return calculable

with Pool(2) as P:
    calculable_list=list(P.map(read_spectr,samples[:10] ))

In [10]:
calculable_list

[True, True, True, True, True, True, True, True, True, True]

# get all Chisq

In [31]:
from command.nexus.NMSSMTools import ReadNToolsOutput
from command.Experiments.Chi_square_Test import Chisq
ignore=[ 'Landau Pole'#27
        ,'Excluded by LUX'
        ,'relic density'
        ,'Muon magn'#37
        ,'b -> c tau nu'#58 always keep alive
        ,'k_bot(H_SM)'
        ,'k_tau(H_SM)'
        ]
def data_forX2(spectr):
    mN=spectr('mass',1000022)
    return{
        'mh':spectr('mass',25)
        ,'bsg':spectr('LOWEN',1)
        ,'bmu':spectr('LOWEN',4)
        ,'omg':spectr('LOWEN',10)
        ,'Psi':(mN,spectr('LOWEN',20))
        ,'Nsd':(mN,spectr('LOWEN',30))
        ,'Psd':(mN,spectr('LOWEN',40))
    }

In [32]:
def read_spectr(sample):
    spectr_dir=sample.documents['spectr']
    SLHAtext=ReadNToolsOutput(spectr_dir,ignore=ignore)
    calculable=not SLHAtext.error
    constraints=SLHAtext.constraints
    if calculable:
        X2=Chisq(**data_forX2(SLHAtext))
    else:
        X2=1E10
    return [calculable,X2,constraints]

In [33]:
from multiprocessing import Pool
with Pool(6) as P:
    X2_list=list(P.map(read_spectr,samples[:10] ))

In [34]:
X2_list

[[True,
  {'mh': 0.47980814134863525,
   'bsg': 3.2561883389202935,
   'bmu': 36.790103736111384,
   'omg': 0.5057421709424116,
   'Psi': 0.0,
   'Nsd': 0.0,
   'Psd': 0.0},
  []],
 [True,
  {'mh': 0.03272135340338278,
   'bsg': 6.357573148690182,
   'bmu': 18.4047415750913,
   'omg': 0.6074831392858236,
   'Psi': 0.0,
   'Nsd': 0.0,
   'Psd': 0.0},
  []],
 [True,
  {'mh': 0.0307736512864619,
   'bsg': 2.4283441629268063,
   'bmu': 4.061989727212622,
   'omg': 4.442018491638444,
   'Psi': 0.0,
   'Nsd': 0.0,
   'Psd': 0.0},
  []],
 [True,
  {'mh': 0.3784639867844519,
   'bsg': 13.235740973386154,
   'bmu': 1.9669983960078399,
   'omg': 1.6677000342329467,
   'Psi': 0.0,
   'Nsd': 0.0,
   'Psd': 0.0},
  []],
 [True,
  {'mh': 0.17057452424854977,
   'bsg': 1.550972917511981,
   'bmu': 5.131021744682012,
   'omg': 0.031128433744929913,
   'Psi': 0.0,
   'Nsd': 0.0,
   'Psd': 0.0},
  []],
 [True,
  {'mh': 0.815806483459878,
   'bsg': 6.931373393894749,
   'bmu': 0.005955525301202513,
   'o

# Followings are not tested and out of date

In [8]:
# test S2P
# from command.NMSSMTools import ReadNMSSMToolsSpectr as readNT
# from command.data_transformer.SpectrumToPandas import SpectrumToPandas as S2P
# spec=readNT(samples[0].documents['spectr'])
# S2P(spec)

In [13]:
# get spectrum DataFrame
from command.data_transformer.SpectrumToPandas import SpectrumToPandas as S2P
spectrums=S2P([sample.spectr for sample in samples])

width= {199}


In [14]:
# mass=spectrums[('spectrum','MASS')][['25','35','36']]
# mass.to_csv('mass.csv')

In [18]:
nothing=[print(i,name_tuple) for i,name_tuple in enumerate(spectrums.columns)]

0 ('spectrum', 'ABUNDANCE', '1', 'unknown')
1 ('spectrum', 'ABUNDANCE', '3', 'unknown')
2 ('spectrum', 'ABUNDANCE', '4', 'unknown')
3 ('spectrum', 'ABUNDANCE', '6', 'unknown')
4 ('spectrum', 'ABUNDANCE', '7', 'unknown')
5 ('spectrum', 'ANNIHILATION', '(5, -5)', 'unknown')
6 ('spectrum', 'ANNIHILATION', '(6, -6)', 'unknown')
7 ('spectrum', 'ANNIHILATION', '(22, 22)', 'unknown')
8 ('spectrum', 'ANNIHILATION', '(22, 23)', 'unknown')
9 ('spectrum', 'ANNIHILATION', '(23, 23)', 'unknown')
10 ('spectrum', 'ANNIHILATION', '(23, 25)', 'unknown')
11 ('spectrum', 'ANNIHILATION', '(24, -24)', 'unknown')
12 ('spectrum', 'ANNIHILATION', "'SigmaV'", 'unknown')
13 ('spectrum', 'EXTPAR', '1', 'M1')
14 ('spectrum', 'EXTPAR', '2', 'M2')
15 ('spectrum', 'EXTPAR', '3', 'M3')
16 ('spectrum', 'EXTPAR', '11', 'Atop')
17 ('spectrum', 'EXTPAR', '12', 'Abottom')
18 ('spectrum', 'EXTPAR', '13', 'Atau')
19 ('spectrum', 'EXTPAR', '16', 'Amuon')
20 ('spectrum', 'EXTPAR', '31', 'L1')
21 ('spectrum', 'EXTPAR', '32', '

In [8]:
# move spectrum files
# for i, s in enumerate(samples[30000:]):
#     new_documents={}
#     new_documents['inp']=os.path.join('./random_samples/record_4/','inp.dat.'+str(i))
#     new_documents['spectr']=new_documents['inp'].replace('inp','spectr')
#     new_documents['omega']=new_documents['inp'].replace('inp','omega')
#     s.MoveTo(new_documents)

In [9]:
#normalize=numpy.linalg.inv(numpy.diag([60,1e3,2e3,6e3,2e3,2e3,2e3,1,1,3e3,3e3,1500]))
# set input point mold
import sys,pandas
sys.path.append('/home/heyangle/Desktop/ScanCraft/ScanCraft')
from command.scan.scan import scan

mold=scan(method='random')
mold.AddScalar('tanB','MINPAR',3,1.,60.)
mold.AddScalar('Atop','EXTPAR'   ,11  ,  -6e3    ,6e3)
mold.AddFollower('Abottom','EXTPAR'   ,12,'Atop')
mold.AddScalar('Lambda','EXTPAR'  ,61  ,1e-3    ,1. ,prior_distribution='exponential')
mold.AddScalar('Kappa','EXTPAR'   ,62 ,1.e-3    ,1. ,prior_distribution='exponential')
mold.AddScalar('A_Lambda','EXTPAR' ,63,-3.e3,3.e3)
mold.AddScalar('A_kappa','EXTPAR' ,64,-3.e3,3.e3)
mold.AddScalar('mu_eff','EXTPAR'  ,65,100.,1500.)

In [10]:
mold.free_parameter_list.keys()

dict_keys(['tanB', 'Atop', 'Lambda', 'Kappa', 'A_Lambda', 'A_kappa', 'mu_eff'])

In [11]:
# get input points
for sample in samples:
    sample.input=copy.deepcopy(mold)
    in_file=sample.documents['inp']
    sample.input.GetValue(in_file)

In [12]:
# from command.data_transformer.InputListToPandas import InputListToPandas as I2P
# input_list=[point.input for point in samples]
# input_table=I2P(input_list,title='accepted')
# input_table[('flags','binary','1/0_is/not','calculable')]=1

In [10]:
samples[0].spectr.NDMCROSSSECT

{1: 0.0, 2: 0.0, 3: 0.0, 4: 0.0}

In [12]:
spectrums#['spectrum']#['NDMCROSSSECT']

0
1
2
3
4
5
6
7
8
9


In [9]:
sorted(samples[0].spectr.__dict__.keys())

['ABUNDANCE',
 'AD',
 'AE',
 'ANNIHILATION',
 'AU',
 'DCINFO',
 'DECAY',
 'DSQMIX',
 'EFFECTIVE_COUPLINGS',
 'ERROR',
 'EXTPAR',
 'FINETUNING',
 'GAUGE',
 'GUTAD',
 'GUTAE',
 'GUTAU',
 'GUTGAUGE',
 'GUTMSOFT',
 'GUTNMSSMRUN',
 'GUTYD',
 'GUTYE',
 'GUTYU',
 'HIGGSBOUNDSINPUTHIGGSCOUPLINGSBOSONS',
 'HIGGSBOUNDSINPUTHIGGSCOUPLINGSFERMIONS',
 'HMIX',
 'LHCCROSSSECTIONS',
 'LHCFIT',
 'LOWEN',
 'LSP',
 'MASS',
 'MINPAR',
 'MODSEL',
 'MSD2',
 'MSE2',
 'MSL2',
 'MSOFT',
 'MSQ2',
 'MSU2',
 'NDMCROSSSECT',
 'NMAMIX',
 'NMHMIX',
 'NMNMIX',
 'NMSSMRUN',
 'RDINFO',
 'REDCOUP',
 'SBOTMIX',
 'SELMIX',
 'SMINPUTS',
 'SPINFO',
 'STAUMIX',
 'STOPMIX',
 'TD',
 'TE',
 'TU',
 'UMIX',
 'USQMIX',
 'VMIX',
 'YD',
 'YE',
 'YU',
 'constraints']

In [17]:
sample.spectr.ANNIHILATION

{(0, 0): 2.0908e-30, (22, 22): 0.089602, (22, 23): 0.9104}

In [36]:
from command.format.block_table import block_table
Mh_list=[]
for mass in [i.spectr.MASS for i in samples]:
    Mh_list.append([mass[25],mass[35],mass[45]])
Mh_array=numpy.array(Mh_list)
Mh_column=pandas.MultiIndex.from_tuples(
    [tuple(['spectr','MASS',i,block_table.MASS(i)]) for i in [25,35,45]]
    ,names=['title','block','code','name']
)
Mh=pandas.DataFrame(Mh_array,columns=Mh_column)

In [49]:
# Mh.to_csv('Higgs_mass.csv')

In [34]:
DM_list=[]
for sample in samples:
    spec=sample.spectr
    if any([i in spec.constraints for i in ['# Lightest neutralino is not the LSP','# Problem in micrOMEGAs']]):
        DM_list.append([numpy.nan]*6)
    else:
        DM_list.append([
            spec.ABUNDANCE[4],spec.NDMCROSSSECT[1],spec.NDMCROSSSECT[2],spec.NDMCROSSSECT[3],spec.NDMCROSSSECT[4],
            spec.ANNIHILATION[(0, 0)]
        ])
DM_array=numpy.array(DM_list)
DM_column=pandas.MultiIndex.from_tuples(
    [('DM','ABUNDANCE','4','relic_density')]+
    [
        tuple(['DM','NDMCROSSSECT',i,'direct_search']) for i in [1,2,3,4]
    ]+
    [('DM','ANNIHILATION',0,'sigmaV')]
)
DM=pandas.DataFrame(DM_array,columns=DM_column)

In [38]:
Bph_list=[]
for sample in samples:
    low=sample.spectr.LOWEN
    Bph_list.append(
        [
            low[1],low[4]
        ]
    )
Bph_array=numpy.array(Bph_list)
Bph_column=pandas.MultiIndex.from_tuples(
    [
        ('Bphysics','LOWEN',1,'b_sg'),
        ('Bphysics','LOWEN',4,'b_mumu')
    ]
)
Bph=pandas.DataFrame(Bph_array,columns=Bph_column)

# os.scandir

In [39]:
import os
with os.scandir('./data0311/') as entries:
    files=list(entries)

In [40]:
f1=files[0]

In [46]:
f1

<DirEntry 'spectr.dat.252055'>

In [42]:
f1.path

'./data0311/spectr.dat.252055'

In [43]:
f1.stat()

os.stat_result(st_mode=33204, st_ino=9355240, st_dev=53, st_nlink=1, st_uid=1000, st_gid=1000, st_size=66205, st_atime=1627872996, st_mtime=1610095150, st_ctime=1622972811)

In [18]:
import pathlib
datadir=pathlib.Path('./data0311/')

In [20]:
datadir

PosixPath('data0311')

In [30]:
f1=list(datadir.iterdir())[0]

In [36]:
f1

PosixPath('data0311/spectr.dat.252055')

In [44]:
with open(f1,'r') as spec:
    text=spec.readlines()

In [45]:
text

['# NMSSMTools OUTPUT IN SLHA FORMAT\n',
 '# Info about spectrum calculator\n',
 'BLOCK SPINFO   # Program information\n',
 '     1   NMSSMTools # Spectrum calculator\n',
 '     2   5.5.2      # Version number\n',
 '     3   # b -> c tau nu more than 2 sigma away (as SM)\n',
 '     3   # k_bot(H_SM) more than 2 sigma away (as SM)\n',
 '     3   # k_tau(H_SM) more than 2 sigma away (as SM)\n',
 '# Input parameters\n',
 'BLOCK MODSEL\n',
 '  3  1   # NMSSM particle content\n',
 '  1  0   # IMOD\n',
 ' 10  0   # ISCAN\n',
 '  9  2   # Call micrOmegas\n',
 '  8  1   # Precision for Higgs masses\n',
 ' 13  1   # Sparticle decays via NMSDECAY\n',
 ' 14  1   # H-> VV,VV*,(V*V*)\n',
 ' 15  7   # Precision for micromegas\n',
 " 16  0   # Extra BLOCK's yes/no\n",
 'BLOCK SMINPUTS\n',
 '     1     1.27920000E+02   # ALPHA_EM^-1(MZ)\n',
 '     2     1.16639000E-05   # GF\n',
 '     3     1.17200000E-01   # ALPHA_S(MZ)\n',
 '     4     9.11870000E+01   # MZ\n',
 '     5     4.21400000E+00   # MB(MB